# Part 4: Enabling users to upload one or more PDF files.

## IMPORTANT: Installation with the exact packages we used
* When you download a full stack app you need to make sure that both backend and frontend use the original packages in order to avoid potential errors caused by installing more modern versions of these packages.
* Since we used pip to install the original backend packages and froze them using pip freeze, you will now use "pip install -r requirements.txt" to install them. Since we also used poetry, you will also use "poetry install".
* Since we used npx to install the original frontend packages, you will now use "npm ci" to install them.
#### Download the code
* Download the code from the github repository.
#### Backend installation
* Since we used both pyenv and poetry to build this project, you will have to use the following approach to install the backend.
* In the terminal, make sure you are in the root directory of the project (v1-165-part4). Pay attention: the root directory of the project and the backend directory have an identic name. Do not mistake them, be sure you are in the root directory of the project now.
* **Create a virtual environment and use pip install to make sure you install the exact same packages we used**:
    * pyenv virtualenv 3.11.4 your-virtual-environment-name
    * pyenv activate your-virtual-environment-name
    * pip install -r requirements.txt
* **Go to the backend directory, create a virtual environment and use poetry install to make sure you install the exact same packages we used**:
    * cd v1-165-part4
    * poetry install
#### Frontend installation
* Open a second terminal window, make sure you are in the root directory of the project (v1-165-part4). Pay attention: the root directory of the project and the backend directory have an identic name. Do not mistake them, be sure you are in the root directory of the project now.
* **Go to the frontend directory, and use npm ci to make sure you install the exact same packages we used**:
    * cd frontend
    * npm ci
#### Ready to go!
* You can now see the code of the app in Visual Studio Code.
* Relax and review the following steps. Remember, since you have pre-installed the modules you will not have to re-install them again.
* **IMPORTANT**: due to the changes we have done in rag_chain.py, if you try to run the backend now you will see an error message. For this lecture, just run the frontend and wait until the following lecture to run the backend.

## Frontend
To enable users to upload one or more PDF files and store them in a directory (`../pdf-documents`), you'll need to modify the React component to include file upload functionality and handle the file upload on the backend server. Note that handling file storage in the `../pdf-documents` directory requires backend logic.

#### Step 1: Add File Input for PDF Uploads

First, in frontend/src/App.tsx add a state to manage the selected files in your React component:

`const [selectedFiles, setSelectedFiles] = useState<FileList | null>(null);`

Next, add an input field for file uploads and a button to submit the files. Update the `return` part of your React component to include these:

```
<div className="p-4 bg-gray-50">
  {/* Existing textarea and send button */}
  
  {/* Add file input for selecting PDF files */}
  <input 
    type="file" 
    accept=".pdf" 
    multiple 
    onChange={(e) => setSelectedFiles(e.target.files)} 
  />
  <button
    className="mt-2 bg-blue-600 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded"
    onClick={handleUploadFiles}
  >
    Upload PDFs
  </button>
</div>
```

#### Step 2: Handle File Uploads

Create a function `handleUploadFiles` to process the file upload when the user clicks the "Upload PDFs" button. This function needs to send the selected files to your backend:

```
const handleUploadFiles = async () => {
  if (!selectedFiles) {
    return;
  }

  const formData = new FormData();
  Array.from(selectedFiles).forEach((file: Blob) => {
    formData.append('files', file);
  });

  // Example: Sending files to a backend endpoint
  try {
    const response = await fetch('http://localhost:8000/upload', {
      method: 'POST',
      body: formData, // No headers for multipart/form-data; fetch adds it automatically
    });
    
    if (response.ok) {
      console.log('Upload successful');
    } else {
      console.error('Upload failed');
    }
  } catch (error) {
    console.error('Error uploading files:', error);
  }
};
```

## Backend Handling
To update your backend code in `server.py` to handle file uploads (specifically PDF documents) and store them in the `../pdf-documents` directory, follow these steps. The changes will involve adding an endpoint to your FastAPI application that can receive and save uploaded files.

#### Step 1: Import Required Modules

First, ensure you import the necessary functions and classes to handle file uploads:

```python
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import RedirectResponse, FileResponse
from fastapi.middleware.cors import CORSMiddleware
from starlette.staticfiles import StaticFiles
import os
import shutil
```

#### Step 2: Create Directory for PDF Documents

Ensure the directory for storing PDF documents exists. You can manually create it or add a check in your code:

```python
pdf_directory = "./pdf-documents"
os.makedirs(pdf_directory, exist_ok=True)
```

#### Step 3: Add an Endpoint for Uploading Files

Add a new endpoint to your FastAPI app that handles file uploads. This endpoint will save the uploaded files to the `../pdf-documents` directory:

```python
@app.post("/upload")
async def upload_files(files: list[UploadFile] = File(...)):
    for file in files:
        try:
            file_path = os.path.join(pdf_directory, file.filename)
            with open(file_path, "wb") as buffer:
                shutil.copyfileobj(file.file, buffer)
        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Could not save file: {e}")
    
    return {"message": "Files uploaded successfully", "filenames": [file.filename for file in files]}
```

#### Complete `server.py` with File Upload Endpoint

After integrating the file upload endpoint, your `server.py` might look something like this:

```python
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import RedirectResponse, FileResponse
from fastapi.middleware.cors import CORSMiddleware
from starlette.staticfiles import StaticFiles
from langserve import add_routes
from app.rag_chain import final_chain
import os
import shutil

app = FastAPI()

# Ensure the PDF documents directory exists
pdf_directory = "../pdf-documents"
os.makedirs(pdf_directory, exist_ok=True)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.mount("/rag/static", StaticFiles(directory="./pdf-documents"), name="static")

@app.get("/")
async def redirect_root_to_docs():
    return RedirectResponse("/docs")

@app.post("/upload")
async def upload_files(files: list[UploadFile] = File(...)):
    for file in files:
        try:
            file_path = os.path.join(pdf_directory, file.filename)
            with open(file_path, "wb") as buffer:
                shutil.copyfileobj(file.file, buffer)
        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Could not save file: {e}")
    
    return {"message": "Files uploaded successfully", "filenames": [file.filename for file in files]}

# Edit this to add the chain you want to add
add_routes(app, final_chain, path="/rag")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
```

This update adds functionality for users to upload one or more PDF files through your FastAPI application, and those files will be stored in the `../pdf-documents` directory relative to the location of `server.py`. Remember to test the upload functionality thoroughly and ensure proper error handling and security measures are in place.


## Load and process data from the UI
To add a button to your UI that triggers the execution of the Python script located at `./rag-data-loader/rag_load_and_process.py`, you'll have to approach this in two main steps: updating your React frontend to include the button and setting up a backend endpoint to execute the Python script. Since React (JavaScript) runs in the browser and cannot directly execute Python scripts on the server or the client's machine, you need an intermediary - your backend server - to run the script.

#### Step 1: Update Your Frontend

First, add a new button in your React component where you want to trigger the script execution. 

```jsx
// Inside your return statement of the React component
<button
  className="mt-2 bg-blue-600 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded"
  onClick={loadAndProcessPDFs}
>
  Load and Process PDFs
</button>
```

Then, define the `loadAndProcessPDFs` function within your component to make a fetch request to the backend endpoint you will create in the next step.

```jsx
const loadAndProcessPDFs = async () => {
  try {
    const response = await fetch('http://localhost:8000/load-and-process-pdfs', {
      method: 'POST',
    });
    if (response.ok) {
      console.log('PDFs loaded and processed successfully');
    } else {
      console.error('Failed to load and process PDFs');
    }
  } catch (error) {
    console.error('Error:', error);
  }
};
```

#### Step 2: Backend Endpoint to Run the Python Script

On your backend server, you'll need to add a new endpoint that, when hit by the frontend, will execute the Python script. Assuming you are using FastAPI as shown in your initial server setup, here’s how you could implement this:

```python
from fastapi import FastAPI
import subprocess

app = FastAPI()

@app.post("/load-and-process-pdfs")
async def load_and_process_pdfs():
    try:
        subprocess.run(["python", "./rag-data-loader/rag_load_and_process.py"], check=True)
        return {"message": "PDFs loaded and processed successfully"}
    except subprocess.CalledProcessError as e:
        return {"error": "Failed to execute script"}
```

#### Notes

- This solution assumes that the server has permission to execute the Python script and access the specified directories and database.
- The `subprocess.run` call in the FastAPI endpoint assumes Python is available in the path where the server runs, and that the script is correctly set up to be executed.

#### Security Considerations

Running scripts or commands from a web application introduces potential security risks. Ensure proper validation and error handling around this functionality. Limit access to such endpoints to authorized users only, and consider the implications of allowing web requests to trigger server-side script execution.